In [84]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
writer_genres = pd.read_csv("../clean data/writer_genres.csv", index_col=0)
writer_genres.head()

,Writer Name,Genres
0,Nicaless,"Fantasy, Young Adult"
1,Rachel B. Moore,"Literary, Literary Fiction"
2,abookishbabe,Young Adult
3,alexabexis,"Romance, Horror/Supernatural, Horror & Superna..."
4,AllYellowFlowers,"Literary, Literary Fiction"


In [45]:
def jaccard(a, b):
    if (type(a) == "float" or type(b) == float):
        return 0
    a = set(a.split(", "))
    b = set(b.split(", "))
    intersect = a.intersection(b)
    union = a.union(b)
    return float(len(intersect)) / len(union)

jaccard(writer_genres['Genres'][0], writer_genres['Genres'][2])

0.5

In [43]:
len(writer_genres)

501

In [85]:
def getSimilar(writer):
    my_genres = writer_genres['Genres'][writer_genres['Writer Name'] == writer][0]
    
    writers = []
    genres = []
    score = []
    for i in range(0, len(writer_genres)):
        writer_name = writer_genres['Writer Name'][i]
        other_genres = writer_genres['Genres'][i]
        writers.append(writer_name)
        genres.append(other_genres)
        score.append(jaccard(my_genres, other_genres))
    df = pd.DataFrame(writers)
    df['genres'] = genres
    df['score'] = score
    df = df[df['score'] != 1.0]
    df = df.sort(['score'], ascending=0)
    
    suggested_genres = df['genres'][0:5]
    
    new_genres = []
    for i in suggested_genres:
        i = i.split(", ")
        for j in i:
            new_genres.append(j)
    print "I suggest you try writing for the following genres:"
    return set(new_genres)

getSimilar("Nicaless")

I suggest you try writing for the following genres:


{'Fantasy', 'Science Fiction', 'Young Adult', 'Young Adult & Youth', 'nan'}